# Mandarin dictionnary


On going. Fixed Python 3 stuff 


In [112]:
import pickle
import re
import codecs

from docx import Document

import jieba as ji
import jieba.analyse as ja

__dict_path = './extra_dict/'
ja.set_stop_words(__dict_path + "stop_words.txt")
ja.set_idf_path("./extra_dict/idf.txt.big");

__dict_path = './extra_dict'

ep = re.compile('a|e|i|o|u')

#I took the html stuff and now it's a string in this module
from html_code import *

In [127]:
class MyWords(dict):
    
    def __init__(self,  
                 src_pkl='words.pkl',
                 tooltip='test'):
        self.ordered_keys = []
        self.src = src_pkl
        tail = ' data-placement="top" title="%s">%s </a>'
        self.wrapper = '<a class="%s" href="#" data-toggle="tooltip"'%tooltip + tail 
        
    
    def tidy(self):
        LL = self.keys()
        LL.sort()
        self.ordered_keys = LL
        
    def show_all(self):
        self.tidy()
        for x in self.ordered_keys:
            print( self.show(x) )
            
    def look_up(self, other):
        return [self.show(x) for x in other if x in self]
    
    def wrap(self, entity):
        if entity in self and self[entity]: 
            payload = '%s : %s' %self[entity]
            return self.wrapper%(payload, entity)
        else:
            return entity
         
    def missing(self, other):
        return [x for x in other if x not in self]
    
    def load(self):
        tt = pickle.load(open(self.src,'rb') )  
        self.update(tt)
        
    def dump(self):
        #not nice but we want to get it back one day
        #even if we have modified the class definition so...
        copy = {}
        copy.update(self)
        with open(self.src, 'wb') as fp:
            pickle.dump(copy, fp)
         
    def show(self, key):
        if not self[key]: return key + ' None'
        payload = ' %s : %s' %self[key]
        if isinstance(payload, str):
            payload = (' %s : %s' %self[key]).decode('utf8')
        return key + payload
    
def txt2web(paras=['',''],
            my_words=[MyWords()],
            fn='test_tool.html'):
    
    '''mark up a text with tool tips
    from a list of  MyWord objects'''
    import html_code
    my_html = []
    for para in paras[:]:
        txt = [x[0] for x in ji.tokenize(para)]
        for word_list in my_words:
            #wrap() returns the string 
            #if it has already been  treated
            txt = [word_list.wrap(w) for w in txt]              
        my_html.append(txt)

    my_html = [ '<p>%s</p>\n'%' '.join( para) for para in my_html]
    my_html = ''.join( my_html)
    my_html= html_wrapper%my_html
    
    with codecs.open( fn, 'w', encoding='utf8') as ff:
        ff.write( my_html)

In [128]:
wiki_words = MyWords(src_pkl='wiki_words.pkl')
wiki_words.load()

In [92]:
#wiki_words

In [129]:
txt2web(contents, my_words=[wiki_words])

In [124]:
wiki_words

{}

In [57]:
tt = MyWords(src_pkl='word_list.pkl',
             tooltip='test1')
tt.load()

In [32]:
#doc = Document('/Users/macbuse/Downloads/ZHU_TRANSLATIONS/holler_CN.docx')
doc = Document('july_trans.docx')

In [130]:
for i, x in enumerate( doc.paragraphs[:]):
    if len(x.text) < 50: continue
    print(i, '-'*60)
    print(x.text)
    

7 ------------------------------------------------------------
非常感谢有这么多人参加今天的交流和对话，我在北京感受到了人生当中非凡的体验。中国是一个拥有悠久历史的伟大国家，我们今天在场的所有人，不仅要作为在中国的中国人为这段历史作出贡献，更要感受到你们是来自世界的中国人。因为我们来自同一个世界。
9 ------------------------------------------------------------
刚才的幻灯片包括从我最初的艺术创作一直到最近的“第三天堂”，“第三天堂”的符号由三个圈组成，是我探索之路的最终呈现。我最早开始探索艺术是在50到60年代，那时我试着寻找自己的艺术定位与身份。在探索的过程中我大量使用自画像——我将镜子当中自己的影像，呈现在画布上。这让我了解自己是谁，要做什么，将来要走向哪里，更重要的是我要为这个世界做什么。我把所有的问题都集中在早期的自画像创作上。
11 ------------------------------------------------------------
镜子是我观照自己的重要手段。一般情况下，艺术家进行自画像创作时把镜子放在画布中间，将镜子中自己的影像在画布上再现出来。我有意把影子变成了画布，画布就是我的镜子。最早的镜画是怎么诞生的呢？我在画布上均匀涂上纯黑的颜料，使其呈现反光的效果，画布便具有镜子的属性。我也从画布中发现了自己的新形象。在凝视像镜子一样反射的黑色画布的过程中，我不仅仅发现了自己，还发现了和我一起观看这个镜面的观众。所以镜面自画像不仅仅是我的自画像，也是所有在画布面前的观众的影像。由此，我自己的自画像变成了世界的自画像。
13 ------------------------------------------------------------
后来我尝试用镜面不锈钢取代全黑的画布，镜面也成为我所有创作最基本的核心要素。这种镜面与观众的关系，在“第三天堂”理念中进一步升华为每一个个体跟世界之间的关系。在镜画的创作过程当中，我发现艺术也可以成为科学。
15 ------------------------------------------------------------
在我看来，镜画不再是一个二维空间的作品，也

In [59]:
result = ji.tokenize(doc.paragraphs[33].text)
for tk in list(result)[:10]:
    print(u"%s\t" % tk[0],)

从	
镜子	
“	
划分	
与	
倍增	
”	
的	
系统	
延伸	


In [60]:
#content = '\n'.join( [para.text for para in doc.paragraphs])

def get_words(txt, topK = 50):  
    tags = ja.extract_tags(txt, topK=topK)
    # if it is a number or english then kill it
    p_kill = re.compile('(\d+)|(a|e|i|o|u)')
    #always convert to lower
    return [x for x in tags if not p_kill.search(x.lower()) ]

contents = [x.text for x in doc.paragraphs]
top100 = get_words(' '.join(contents))
top100

['镜子',
 '我们',
 '一个',
 '社会',
 '天堂',
 '第三',
 '自己',
 '影像',
 '这个',
 '镜画',
 '镜面',
 '画布',
 '人类',
 '自画像',
 '无限',
 '两个',
 '当中',
 '符号',
 '所有',
 '而是',
 '一种',
 '存在',
 '代表',
 '民主',
 '实践',
 '中国',
 '过程',
 '不断',
 '不同',
 '两面',
 '人民',
 '领域',
 '探索',
 '创作',
 '所以',
 '这种',
 '产生',
 '自然',
 '政治',
 '艺术',
 '反射',
 '世界',
 '组成',
 '呈现',
 '个体',
 '这一',
 '变化',
 '现在',
 '生活',
 '实现']

In [62]:
mm = wiki_words.missing(top100) 
print( '\ '.join( tt.missing(mm) ) )

NameError: name 'wiki_words' is not defined

In [412]:
tokens = get_words(' '.join(paras) )
for x in tt.look_up(tokens):
    print x

一种 yī zhǒng : one kind of, one type of
这个 zhè ge : this, this one
方式 fāng shì : way, manner, style, mode, pattern, 
这种 zhè zhǒng : this, this kind of, this sort of, this type of
系列 xì liè : series, set
过程 guò chéng : course of events, process, 
抽象 chōu xiàng : abstract, abstraction
一个 xià yī ge : the next one
我们 wǒ men : we, us, ourselves, our
发展 fā zhǎn : development, growth, to develop, to grow, to expand
自我 zì wǒ : self-, ego (psychology)
作品 zuò pǐn : work (of art), opus, 
所以 suǒ yǐ : therefore, as a result, so, the reason why
中国 Zhōng guó : China
线条 xiàn tiáo : line (in drawing, calligraphy etc), the lines or contours of a three-dimensional object (hairstyle, clothing, car etc)
这些 zhè xiē : these
只有 zhǐ yǒu : only
可能 kě néng : might (happen), possible, probable, possibility, probability, maybe, perhaps
展览 zhǎn lǎn : to put on display, to exhibit, exhibition, show, 
传统 chuán tǒng : tradition, traditional, convention, conventional
早期 zǎo qī : early period, early phase, early stage
当代

In [414]:
print ' '.join(tt.missing(tokens) )

张帆 绘画 布面 劳作 急就章 自叙 属于 成为 山林 现在 逻辑 危险 视为 奇观 基座 气息 仍然 形式主义 水图 抽象画 一张 书写 丙烯 质感 油画 采薇 符号 缓慢 似乎 高度 引自 僵尸 命题 苏轼 废品 存在 几个 有着 古典 几乎 真的 境界 身边 某个 容易 变化 格子 今格 设想 还是 消失 国际 画面 摹拟 之三 它们 完成 占据 禁锢


# Data extractors

### Sources for vocabulary : 

1. https://en.wiktionary.org/wiki/
1. https://chinese.yabla.com

Neither of these are difficult to scrape

#### Test texts :

1. art.china.cn
1. mp.weixin.qq.com

Neither of these are difficult to scrape
but they use different container classes...see code


### Wiki

The big problem here is that there are repeated <a></a> tags
and the regexp isn't trivial

`ur'<a.*?>(.*?)</a></span>.*?<a.*?</a></span>.*?<a.*?>(.*?)</a>\)(.*?)<' `

### Yabla

This is a standard data collector :

1. should check the current dictionnaries before making a request
2. the utf8 has to be encoded for the request string



In [106]:
import urllib2

def yabla_lookup(word_bag = []):
    pdef =  re.compile(ur'class="definition">.*?="pinyin">(.*?)<.*?="meaning">(.*?)<',re.DOTALL)
    stuff = {x:'' for x in word_bag} 

    for word in word_bag:
        print word,
        qq = urllib2.quote(word.encode('utf8'))
        base_url = u'https://chinese.yabla.com/chinese-english-pinyin-dictionary.php?define='
        data = urllib2.urlopen(base_url + qq).read()
        stuff[word] = pdef.search(data)
    return stuff

def clean(txt, meaning=False):
    if meaning : return txt.split('\n')[0] + ' '
    pinyin, meaning = y.group(1), y.group(2)
    meanings = meaning.split('CL:')[0] 
    #there is a trailing \n so strip() it 
    return (pinyin, meaning.strip().replace('\n',', ') )

In [ ]:
import urllib2, re

dp = ur'<a.*?>(.*?)</a></span>.*?<a.*?</a></span>.*?<a.*?>(.*?)</a>\)(.*?)<'
dp = re.compile(dp)

base_url = 'https://en.wiktionary.org/wiki/Appendix:Mandarin_Frequency_lists'

n = 4
data = urllib2.urlopen(base_url + '/%d001-%d000'%(n,n+1)).read()
matches = dp.findall(data)

for x in matches[:]:
    entity, pinyin, meaning = [unicode(w,'utf8') for w in x]
    wiki_words[entity = (pinyin, meaning[3:])

In [ ]:
from bs4 import BeautifulSoup
import urllib2

#data = urllib2.urlopen('http://art.china.cn/tongzhi/2018-03/13/content_40249473.htm').read()

#data = urllib2.urlopen('http://art.china.cn/ysjt/2017-09/20/content_40022118.htm').read()
#data = urllib2.urlopen('http://art.china.cn/haiwai/2016-05/06/content_8753835.htm').read()
data = urllib2.urlopen('http://mp.weixin.qq.com/s/FIBpmy2GFPRMvXFNCd8lLg').read()

def extract_contents(html,
                        div_class='content'):
    
    '''return all the paragraphs in the html'''
    
    soup = BeautifulSoup(html, 'lxml')
    payload = soup.find('div', {'class' : div_class})
    return [p.text for p in payload.find_all('p')]  

class4 = {'art.china.cn': 'content', 'mp.weixin.qq.com' : "rich_media_content " }
contents2 = extract_contents(data, 
                             div_class=class4['mp.weixin.qq.com'])

In [55]:
new_stuff = []
tt.update(new_stuff)
tt.dump()
mm = wiki_words.missing(top100) 
print('\ '.join( tt.missing(mm)) )
new_stuff = yabla_lookup(tt.missing(mm)) 

NameError: name 'file' is not defined

In [108]:
failed = []
for x,y in new_stuff.items():
    try:
        new_stuff[x] = clean(y)
    except:
        failed.append(x)
        
print '\n'.join(failed)
    

山林
一张
布面
急就章
水图
张帆
自叙
抽象画


So with what is left we are going to have to do some **mechanical turk**

Trouble is that google translate uses seriously obfuscated javascript....

In [109]:
src = u'''山林
采薇
引自
一张
真的
布面
今格
水图
自叙
抽象画'''

pyn = u'''Shānlín
cǎi wēi
yǐn zì
yī zhāng
zhēn de
bù miàn
jīn gé
shuǐ tú
zìxù
chōuxiàng huà'''

tgt = '''Mountain forest
Petch
cited from
One sheet
Really
Cloth surface
In this case
Water map
Self-description
Abstract painting'''

payload = zip([ x.lower() for x in pyn.split('\n') ],
[ x.lower() for x in tgt.split('\n') ])
for w, dd in zip( src.split('\n'), payload):
    new_stuff[w] = dd


Keep the dictionnary clean !! 
There is a risk of getting str from the clipboard 

In [419]:
for x in new_stuff.keys():
    if isinstance(x,str):
        del stuff[x]

for x in [u'苏摩 苏摩']:
    if x in new_stuff:
        del new_stuff[x]

In [147]:
for x,y in new_stuff.items():
    print x, '%s %s' %y

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(x, '%s %s' %y)? (<ipython-input-147-f9e3c6647361>, line 2)

In [25]:
def multiple_replace(text, adict):
    rx = re.compile('|'.join(map(re.escape, adict)))
    def one_xlat(match):
        return adict[match.group(0)]
    return rx.sub(one_xlat, text)

In [248]:
#multiple_replace(content, m2ex)

In [49]:
content2 = '\n'.join(x.text for x in doc.paragraphs 
                             if x.text and len(ep.findall(x.text) ) < 30 )
#print content2

[ len(ep.findall(x.text) )for x in doc.paragraphs  if x.text] 


[7, 0, 20, 390, 0, 222, 5, 275, 355, 0, 186, 5]

In [52]:
[ len(ep.findall(x.text)) / float(len(x.text)) for x in doc.paragraphs  if x.text]

[0.175,
 0.0,
 0.055865921787709494,
 0.31630170316301703,
 0.0,
 0.2817258883248731,
 0.01845018450184502,
 0.29411764705882354,
 0.281076801266825,
 0.0,
 0.2957074721780604,
 0.18518518518518517]

In [89]:
paras = [ x for x in contents2 if len(x)>0 and len(ep.findall(x.lower()))/float(len(x)) <.2 ]

In [241]:
new_words = get_words(contents2)
for x in tt.look_up(new_words)[:10]:
    print x


观众 guān zhòng : spectators, audience, visitors (to an exhibition etc)
装置 zhuāng zhì : to install, installation, equipment, system, unit, device
一个 xià yī ge : the next one
作品 zuò pǐn : work (of art), opus, 
空间 kōng jiān : space, 
展览 zhǎn lǎn : to put on display, to exhibit, exhibition, show, 
选择 xuǎn zé : to select, to pick, choice, option, alternative
艺术家 yì shù jiā : artist, 
自然 zì rán : nature, natural, naturally
思考 sī kǎo : to reflect on, to ponder over


In [246]:
new_stuff = yabla_lookup(tt.missing(new_words))

 奥莱 卡斯特 渐近线 能够 决定 无限 怀疑 幻觉 还有 观展 其中 感官 人们 其他 直线 这样 不同 曲线 参与 哪条 愉悦 当代艺术 颜色 文森特 时间 融入 这位 看来 根据 延伸 生于 组成 视觉 应该 所以 只有 可能 不断 相同 展现 比利 才能 对称 固有 拥有 这次 现长 别人 直接 对比 参展者 近于零 冥想 彩色 比如说 觉得 二分之一


In [256]:
failed = []
for x,y in new_stuff.items():
    try:
        new_stuff[x] = clean(y)
    except:
        failed.append(x)

In [251]:
print '\ '.join(failed)

融入\ 近于零\ 还有\ 这位\ 看来\ 感官\ 文森特\ 展现\ 觉得\ 观展\ 怀疑\ 时间\ 视觉\ 卡斯特\ 颜色\ 所以\ 能够\ 根据\ 可能\ 冥想\ 只有\ 其他\ 参展者\ 人们\ 决定\ 彩色\ 不断\ 参与\ 其中\ 二分之一\ 比如说\ 比利\ 延伸\ 对称\ 应该\ 奥莱\ 哪条\ 组成\ 渐近线\ 直线\ 拥有\ 固有\ 相同\ 生于\ 才能\ 不同\ 曲线\ 现长\ 这次\ 无限\ 当代艺术\ 别人\ 幻觉\ 直接\ 对比\ 这样\ 愉悦


In [522]:
tt.update(new_stuff)

In [425]:
tt.dump()
tt = MyWords()
tt.load()

In [367]:
print '\n'.join( tt.missing(tags2))

蒋兆和
于洋
苦茶
生民
乡关
家国


## Automatic translation using google translate


In [ ]:
from googletrans import Translator
translator = Translator()

target = Document()

target.add_heading('Translation', 0)

data = []
for x in doc.paragraphs[:]:
    if not x.text  : continue
    target.add_paragraph( x.text  )
    x_clean = x.text.replace('??','')
    x_clean = x_clean.replace('**','')
    gt = translator.translate(x_clean, dest='en')
    target.add_paragraph( gt.text )
    target.add_paragraph( '-'*60)
    
target.save('my_trans.docx')



In [145]:
from googletrans import Translator
translator = Translator()

def txt2web2(paras=['',''],
            my_words=[MyWords()],
            fn='test_tool.html'):
    
    '''mark up a text with tool tips
    from a list of  MyWord objects'''
    import html_code
    target = []

    for para in paras:

        x_clean = para.replace('??','')
        x_clean = x_clean.replace('**','')
        txt = [x[0] for x in ji.tokenize(para)]
        
        for word_list in my_words:
            #wrap() returns the string 
            #if it has already been  treated
            txt = [word_list.wrap(w) for w in txt]              
        target.append(txt)
        gt = translator.translate(x_clean, dest='en')
        target.append( gt.text )
        target.append( '-'*60)

    my_html = [ '<p>%s</p>\n'%' '.join( para) for para in target]
    my_html = ''.join( my_html)
    my_html= html_wrapper%my_html
    
    with codecs.open( fn, 'w', encoding='utf8') as ff:
        ff.write( my_html)

In [146]:
txt2web2(paras=[x.text for x in doc.paragraphs],my_words=[wiki_words])

In [135]:
[x.text for x in doc.paragraphs]

['米开朗基罗·皮斯特莱托 MICHELANGELO PISTOLETTO | CAFA讲座文本',
 '',
 '主题：你和我等于三',
 '时间：2018年6月24日下午',
 '地点：中央美院美术馆学术报告厅',
 '',
 '    ',
 '非常感谢有这么多人参加今天的交流和对话，我在北京感受到了人生当中非凡的体验。中国是一个拥有悠久历史的伟大国家，我们今天在场的所有人，不仅要作为在中国的中国人为这段历史作出贡献，更要感受到你们是来自世界的中国人。因为我们来自同一个世界。',
 '',
 '刚才的幻灯片包括从我最初的艺术创作一直到最近的“第三天堂”，“第三天堂”的符号由三个圈组成，是我探索之路的最终呈现。我最早开始探索艺术是在50到60年代，那时我试着寻找自己的艺术定位与身份。在探索的过程中我大量使用自画像——我将镜子当中自己的影像，呈现在画布上。这让我了解自己是谁，要做什么，将来要走向哪里，更重要的是我要为这个世界做什么。我把所有的问题都集中在早期的自画像创作上。',
 '',
 '镜子是我观照自己的重要手段。一般情况下，艺术家进行自画像创作时把镜子放在画布中间，将镜子中自己的影像在画布上再现出来。我有意把影子变成了画布，画布就是我的镜子。最早的镜画是怎么诞生的呢？我在画布上均匀涂上纯黑的颜料，使其呈现反光的效果，画布便具有镜子的属性。我也从画布中发现了自己的新形象。在凝视像镜子一样反射的黑色画布的过程中，我不仅仅发现了自己，还发现了和我一起观看这个镜面的观众。所以镜面自画像不仅仅是我的自画像，也是所有在画布面前的观众的影像。由此，我自己的自画像变成了世界的自画像。',
 '',
 '后来我尝试用镜面不锈钢取代全黑的画布，镜面也成为我所有创作最基本的核心要素。这种镜面与观众的关系，在“第三天堂”理念中进一步升华为每一个个体跟世界之间的关系。在镜画的创作过程当中，我发现艺术也可以成为科学。',
 '',
 '在我看来，镜画不再是一个二维空间的作品，也不是三维空间的作品，而是四维空间的作品，包含时间这一要素。因为镜中的影像仅仅是观看瞬间的影像，这个影像具有时间性，并不断变化。因为看镜子是一种直接体验，眼前所看到的影像只存在于现在，回头看的时候便不复存在，不看它的时候又变成了将来状态。这种不断变化、不断更新的“现在”的时间维度，也是无限的一种表现。镜画中的影像